In [12]:
import argparse
import collections
import re
import spacy
from tqdm import tqdm
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [7]:
def load_corpus(path):
    print('Loading corpus...')
    corpus = {}
    n_text = 0
    with open(path, errors='ignore') as f:
        all_text = ' '.join(f.read().split())
        for raw_text in tqdm(all_text.split('</DOC>')):
            if not raw_text:
                continue
            result = re.search(r'\<DOCNO\>(.*)\<\/DOCNO\>', raw_text)
            if not result:
                continue
            doc_id = result.group(1)
            doc_id = doc_id.strip()

            result = re.search(r'\<TEXT\>(.*)\<\/TEXT\>', raw_text)
            doc_text = ''
            if result:
                doc_text = result.group(1)
                doc_text = BeautifulSoup(doc_text, 'html.parser').text
                doc_text = doc_text.strip()
                if doc_text:
                    n_text += 1

            corpus[doc_id] = ' '.join(doc_text.split())

    print(f'Loaded {len(corpus)} docs, {n_text} with texts.')
    return corpus

In [13]:
def load_corpus(path):
    print('Loading corpus...')
    corpus = {}
    n_text = 0
    with open(path, errors='ignore') as f:
        all_text = ' '.join(f.read().split())
        for raw_text in tqdm(all_text.split('</DOC>')):
            if not raw_text:
                continue
            result = re.search(r'\<DOCNO\>(.*)\<\/DOCNO\>', raw_text)
            if not result:
                continue
            doc_id = result.group(1)
            doc_id = doc_id.strip()

            result = re.search(r'\<TEXT\>(.*)\<\/TEXT\>', raw_text)
            doc_text = ''
            if result:
                doc_text = result.group(1)
                doc_text = doc_text.replace('<P>', ' ').replace('</P>', ' ')
                doc_text = doc_text.strip()
                if doc_text:
                    n_text += 1

            corpus[doc_id] = ' '.join(doc_text.split())

    print(f'Loaded {len(corpus)} docs, {n_text} with texts.')
    return corpus

In [14]:
corpus = load_corpus('repos/pygaggle/data/robust04/trec_disks_4_and_5_concat.txt')

Loading corpus...


100%|██████████| 528160/528160 [00:58<00:00, 8980.82it/s] 


Loaded 528156 docs, 523869 with texts.


In [9]:
df = pd.read_csv('repos/pygaggle/data/robust04/run.robust04.bm25.txt', sep=' ', header=None)
bm25_docs = df.iloc[:,2].to_list()
empty_docs = [i for i in corpus.keys() if corpus[i]=='']
len([i for i in empty_docs if i in bm25_docs])

0

In [4]:
from bs4 import BeautifulSoup
doc_id = list(corpus.keys())[np.random.randint(len(corpus.keys()))]
doc_text = corpus[doc_id]
soup = BeautifulSoup(doc_text, 'html.parser')

In [5]:
doc_text

'Net asset value of London and St Lawrence Investment stood at 138.24p at February 29, down from 140.02p a year earlier. Net revenue for the six months to end-February amounted to Pounds 520,684 (Pounds 493,338) for earnings of 2.74p (2.6p) per share.'

In [35]:
doc_id

'FR940817-2-00063'

In [ ]:
corpus['FBIS3-20511']

In [38]:
[key for key, value in corpus.items() if '---------------' in value][7000:7010]

['FT923-8736',
 'FT923-8742',
 'FT923-8784',
 'FT923-8838',
 'FT923-8875',
 'FT923-8885',
 'FT923-8892',
 'FT923-8976',
 'FT923-8980',
 'FT923-9005']

In [ ]:
[print(corpus[i]+'\n\n\n\n\n') for i in [key for key, value in corpus.items() if '---------------' in value][:10]]

In [9]:
BeautifulSoup(corpus['FT931-16924'], 'html.parser').text

'-------------------------------- COMPANIES IN THIS ISSUE -------------------------------- UK Allied Breweries 36 BTR 31 Brent Walker 36 Cable & Wireless 17 Forte 36 Framgord 32 GKN 32 GRE 29 Greencore 32 Hammerson 36 ICI 30 Maple Leaf Inns 36 National Breakdown 32 Relica Food 32 Savoy Hotel 36 Scottish Equitable 32 Shell 32 TIP Europe 30 Tiphook 30 Overseas ABB 30 Alcatel Alsthom 30 Candy 30 Carlsberg 32 EDS 36 Fiat 8 Fininvest 31 Finmeccania 32 GSI 32 General Motors 36 Goldman Sachs 37 Heineken 30 Interbrew 31 KLM 30 Labatt 36 MBP Software 36 Max Mara 32 McDonalds 8 Monsanto Europe 32 Peoples Jewellers 37 Peugoet 30 Renault 8 Siemens 32 Solvay 31 Sony Europe 30 Viscofan 30 Volkswagen 8 Wurttembergische 32 --------------------------------'